<a href="https://colab.research.google.com/github/Ayushraj6942/GDG-Project/blob/main/GDG_group_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install requests yfinance newsapi-python pymupdf transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 43.2 MB/s eta 0:00:00


In [16]:
import yfinance as yf
import requests
from newsapi import NewsApiClient
from transformers import pipeline

# API Keys
FMP_API_KEY = "YOUR_FMP_API_KEY"  # Replace with your Financial Modeling Prep API key
NEWS_API_KEY = "e0f0a752886144fca3fedfe24e14dd3dbb"  # Replace with your NewsAPI key

# Initialize Sentiment Analysis Pipeline
sentiment_pipeline = pipeline("sentiment-analysis", device="cpu")

# Function to fetch historical stock prices
def get_historical_prices(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period="5d")
        return data
    except Exception as e:
        print(f"Error fetching historical prices for {ticker}: {e}")
        return None

# Function to fetch fundamental ratios
def get_fundamental_ratios(ticker):
    try:
        url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey={FMP_API_KEY}"
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching ratios for {ticker}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching ratios for {ticker}: {e}")
        return None

# Function to fetch news articles
def get_stock_news(query):
    try:
        newsapi = NewsApiClient(api_key=NEWS_API_KEY)
        news = newsapi.get_everything(q=query, language="en", sort_by="relevancy", page_size=5)
        return news['articles'] if 'articles' in news else []
    except Exception as e:
        print(f"Error fetching news for {query}: {e}")
        return []

# Main workflow
def analyze_stock(ticker):
    print(f"\nFetching data for {ticker}...")

    # Fetch and print historical prices
    historical_prices = get_historical_prices(ticker)
    if historical_prices is not None:
        print(f"\nHistorical Prices for {ticker}:")
        print(historical_prices)

    # Fetch and print fundamental ratios
    fundamental_ratios = get_fundamental_ratios(ticker)
    if fundamental_ratios:
        print(f"\nFundamental Ratios for {ticker}:")
        print(fundamental_ratios[:3])  # Print the first 3 ratios for brevity

    # Fetch and analyze news articles
    news_articles = get_stock_news(ticker)
    print(f"\nLatest News for {ticker}:")
    for article in news_articles:
        title = article.get('title', 'No Title')
        description = article.get('description', 'No Description')

        print(f"- {title}: {description}")

        # Perform sentiment analysis only if description is valid
        if description and description != "No Description":
            try:
                sentiment = sentiment_pipeline(description)[0]
                print(f"  Sentiment: {sentiment['label']} (Score: {sentiment['score']})")
            except Exception as e:
                print(f"  Error in sentiment analysis: {e}")
        else:
            print("  Sentiment: No description available for analysis.")

# List of stocks to analyze
stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "NFLX", "BRK-B", "JNJ"]

# Run analysis for each stock
for stock in stocks:
    analyze_stock(stock)



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu



Fetching data for AAPL...

Historical Prices for AAPL:
                                 Open        High         Low       Close  \
Date                                                                        
2025-01-17 00:00:00-05:00  232.119995  232.289993  228.479996  229.979996   
2025-01-21 00:00:00-05:00  224.000000  224.419998  219.380005  222.639999   
2025-01-22 00:00:00-05:00  219.789993  224.119995  219.789993  223.830002   
2025-01-23 00:00:00-05:00  224.740005  227.029999  222.300003  223.660004   
2025-01-24 00:00:00-05:00  224.779999  225.630005  221.410004  222.779999   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-01-17 00:00:00-05:00  68488300        0.0           0.0  
2025-01-21 00:00:00-05:00  98070400        0.0           0.0  
2025-01-22 00:00:00-05:00  64126500        0.0           0.0  
2025-01-23 00:00:00-05:00  60234800        0.0           0.0  
2025-01-24 00:00:00-05:00 

In [23]:
import yfinance as yf
import requests
import numpy as np
import pandas as pd
from newsapi import NewsApiClient
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# API Keys
FMP_API_KEY = "q0qxT3uhrMzldjJkpFJepqzj6lR3DoeK"  # Replace with your Financial Modeling Prep API key
NEWS_API_KEY = "e0f0a752886144fca3fedfe24e14dd3d"  # Replace with your NewsAPI key

# Initialize Sentiment Analysis Pipeline
sentiment_pipeline = pipeline("sentiment-analysis", device="cpu")

# Function to fetch historical stock prices
def get_historical_prices(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period="1y")
        return data
    except Exception as e:
        print(f"Error fetching historical prices for {ticker}: {e}")
        return None

# Function to fetch fundamental ratios
def get_fundamental_ratios(ticker):
    try:
        url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey={FMP_API_KEY}"
        response = requests.get(url)
        if response.status_code == 200:
            return pd.DataFrame(response.json())
        else:
            print(f"Error fetching ratios for {ticker}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching ratios for {ticker}: {e}")
        return None

# Function to fetch news articles
def get_stock_news(query):
    try:
        newsapi = NewsApiClient(api_key=NEWS_API_KEY)
        news = newsapi.get_everything(q=query, language="en", sort_by="relevancy", page_size=5)
        return news['articles'] if 'articles' in news else []
    except Exception as e:
        print(f"Error fetching news for {query}: {e}")
        return []

# Function to collect sentiment scores
def get_sentiment_score(description):
    if description and description.strip():
        sentiment = sentiment_pipeline(description)[0]
        return 1 if sentiment['label'] == 'POSITIVE' else -1
    return 0

# Data collection and feature engineering
# Data collection and feature engineering
# Data collection and feature engineering with better handling of missing data
def collect_data(tickers):
    all_data = []
    for ticker in tickers:
        print(f"Collecting data for {ticker}...")

        # Historical prices
        historical_data = get_historical_prices(ticker)
        if historical_data is not None:
            historical_data['Ticker'] = ticker
            historical_data['Date'] = historical_data.index
            historical_data.reset_index(drop=True, inplace=True)

        # Fundamental ratios
        fundamental_data = get_fundamental_ratios(ticker)
        if fundamental_data is not None:
            print(f"Fundamental Data for {ticker}:")
            print(fundamental_data.head())  # Inspect fundamental data
            fundamental_data = fundamental_data[['date', 'currentRatio', 'debtEquityRatio', 'returnOnEquity']]
            fundamental_data['Ticker'] = ticker

        # News sentiment
        news_articles = get_stock_news(ticker)
        sentiment_scores = [get_sentiment_score(article.get('description', '')) for article in news_articles]
        avg_sentiment = np.mean(sentiment_scores) if sentiment_scores else 0

        # Convert 'Date' to a naive datetime (without timezone)
        if historical_data is not None:
            historical_data['Date'] = historical_data['Date'].dt.tz_localize(None)

        # Convert 'date' to datetime if it's not already
        if fundamental_data is not None:
            fundamental_data['date'] = pd.to_datetime(fundamental_data['date'])

        # Merge data
        if historical_data is not None and fundamental_data is not None:
            merged_data = historical_data.merge(fundamental_data, left_on='Date', right_on='date', how='left')
            merged_data['Sentiment'] = avg_sentiment
            all_data.append(merged_data)

    # Concatenate all the collected data into one DataFrame
    full_data = pd.concat(all_data, ignore_index=True) if all_data else None

    # Inspect the full data and check how much data we have
    if full_data is not None:
        print("Merged Data Sample:")
        print(full_data.head())
        print(f"Total Rows: {len(full_data)}")

        # Fill missing values for fundamental data (e.g., forward fill)
        full_data[['currentRatio', 'debtEquityRatio', 'returnOnEquity']] = full_data[['currentRatio', 'debtEquityRatio', 'returnOnEquity']].fillna(method='ffill')

        print("Data after filling missing values:")
        print(full_data.head())

    return full_data

# Train and evaluate a model with better handling of missing data
def train_model(data):
    # Prepare features and target
    data['Target'] = data['Close'].shift(-1)  # Predict next day close price
    data.dropna(inplace=True)

    # Check if there's enough data
    if len(data) <= 1:
        print("Not enough data to train the model.")
        return None

    features = ['Open', 'High', 'Low', 'Volume', 'currentRatio', 'debtEquityRatio', 'returnOnEquity', 'Sentiment']
    X = data[features]
    y = data['Target']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Check if the split is valid
    if len(X_train) == 0 or len(X_test) == 0:
        print("Not enough data after splitting. Reducing test size.")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate model
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Model Mean Squared Error: {mse}")

    return model

# Main workflow
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "NFLX", "BRK-B", "JNJ"]
data = collect_data(tickers)

if data is not None:
    print("Training the model...")
    model = train_model(data)
else:
    print("No data collected. Please check your API keys and connectivity.")



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Fundamental Data for AAPL:
  symbol        date calendarYear period  currentRatio  quickRatio  cashRatio  \
0   AAPL  2024-09-28         2024     FY      0.867313    0.826007   0.169753   
1   AAPL  2023-09-30         2023     FY      0.988012    0.944442   0.206217   
2   AAPL  2022-09-24         2022     FY      0.879356    0.847235   0.153563   
3   AAPL  2021-09-25         2021     FY      1.074553    1.022115   0.278449   
4   AAPL  2020-09-26         2020     FY      1.363604    1.325072   0.360710   

   daysOfSalesOutstanding  daysOfInventoryOutstanding  operatingCycle  ...  \
0               61.832560                   12.642571       74.475130  ...   
1               58.075649                   10.791292       68.866941  ...   
2               56.400205                    8.075698       64.475903  ...   
3               51.390969                   11.276593       62.667561  ...   
4               49.787534                    8.741883       58.529418  ...   

   priceToSalesRa

<ipython-input-23-ed2a21c4aaf0>:111: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  full_data[['currentRatio', 'debtEquityRatio', 'returnOnEquity']] = full_data[['currentRatio', 'debtEquityRatio', 'returnOnEquity']].fillna(method='ffill')
